In [1]:
!pip install transformers datasets trl accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM


model_name = "EleutherAI/pythia-410m-deduped"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/911M [00:00<?, ?B/s]

The `GPTNeoXSdpaAttention` class is deprecated in favor of simply modifying the `config._attn_implementation`attribute of the `GPTNeoXAttention` class! It will be removed in v4.48


In [3]:
from transformers import AutoModelForSequenceClassification

reward_model_name = "facebook/roberta-hate-speech-dynabench-r4-target"
reward_model = AutoModelForSequenceClassification.from_pretrained(reward_model_name)


config.json:   0%|          | 0.00/816 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [4]:
from datasets import load_dataset

dataset = load_dataset("allenai/real-toxicity-prompts")


README.md:   0%|          | 0.00/4.22k [00:00<?, ?B/s]

prompts.jsonl:   0%|          | 0.00/67.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/99442 [00:00<?, ? examples/s]

In [11]:
from trl import PPOTrainer, PPOConfig

ppo_config = PPOConfig(
    learning_rate=1.41e-5,
    batch_size=16,
    output_dir="./ppo_results"
)
ppo_trainer = PPOTrainer(
    model=model,
    ref_model=model,
    tokenizer=tokenizer,
    dataset=dataset["train"],
    config=ppo_config
)


<ipython-input-11-01f8271bfb43>:8: FutureWarning: `config` is deprecated and removed starting from version 0.15.0 for `PPOTrainer.__init__`. Use `args` instead.
  ppo_trainer = PPOTrainer(
/usr/local/lib/python3.11/dist-packages/transformers/utils/deprecation.py:165: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.15.0 for `PPOTrainer.__init__`. Use `processing_class` instead.
  return func(*args, **kwargs)


TypeError: PPOTrainer.__init__() got an unexpected keyword argument 'dataset'

In [7]:
import torch

def compute_rewards(samples, reward_model, tokenizer):
    inputs = tokenizer(samples, return_tensors="pt", truncation=True, padding=True)
    outputs = reward_model(**inputs)
    scores = torch.softmax(outputs.logits, dim=-1)[:, 1]  # Toxicity probabilities
    return -scores  # Negative score for detoxification


In [ ]:
for epoch in range(ppo_config.epochs):
    for batch in dataset["train"].shuffle().select(range(ppo_config.batch_size)):
        # responses
        queries = tokenizer(batch["prompt"], return_tensors="pt", truncation=True, padding=True).input_ids
        responses = [tokenizer.decode(output) for output in model.generate(queries, max_length=50)]

        # rewards
        rewards = compute_rewards(responses, reward_model, tokenizer)

        # Train with PPO
        ppo_trainer.step(queries, responses, rewards)


In [ ]:
model.save_pretrained("pythia-410m-deduped-detoxified")
tokenizer.save_pretrained("pythia-410m-deduped-detoxified")
